<a href="https://colab.research.google.com/github/Kairory/ML-python_study/blob/main/%D0%93%D0%BB%D0%B0%D0%B2%D0%B0_4_%D0%9F%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

Аугментация данных

Уменьшить переобучение модели можно с помощью дополнительных обучающих данных. Аугментация («расширение», «дополнение») данных — это метод искусственного увеличения количества данных. Новые образцы данных создаются посредством различных преобразований уже имеющихся данных — переворачиванием, масштабированием, вращением, срезами и прочими манипуляциями.

In [ ]:
aug = tf.keras.Sequential(tf.keras.layers.RandomFlip("horizontal"))
# Агументация во время обучение
aug_model = tf.keras.Sequential()
aug_model.add(aug)
# . . .

Аугментация до обучения

Аугментацию можно применять отдельно к набору данных во время предва-
рительной обработки изображений. В таком случае она может выполняться
на CPU, в то время как обучение основной модели асинхронно проводится
на GPU.

In [ ]:
inputs = tf.keras.Input((28, 28, 3))
conv_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding="same",
                                kernel_regularizer="L1L2")(inputs)
                                # kernel_regularizer=tf.keras.regularizers.L1L2(0.01, 0.01))(inputs) для более ручной настройки
bn_1 = tf.keras.layers.BatchNormalization()(conv_1)
act_1 = tf.keras.layers.Acrivation("relu")(bn_1)
pool_1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding="same")(act_1) # по умолчание strides = poool_size
do_1 = tf.keras.layers.Dropout(0.6)(pool_1) # Отключает нейроны только во время обучения


In [ ]:
# Наборы данных
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_ds = train_ds.batch(32).map(lambda x,y:(augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(32)

Наборы данных

In [ ]:
ds = tf.data.Dataset.list_files("/path/to/csv/files/*.csv")

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

image_directory = "./input_data/images/"

train_dataset = image_dataset_from_directory(directory=image_directory,
                                             label_mode="int",
                                             validation_split=0.3,
                                             subset="training",
                                             image_size=(28,28),
                                             batch_size=64)

validation_dataset = image_dataset_from_directory(directory=image_directory,
                                                  label_mode="int",
                                                  validation_split=0.3,
                                                  subset="validation",
                                                  image_size=(28,28),
                                                  batch_size=64)



Также нужно упомянуть о режиме label_mode. Когда параметр label_mode равен int, это значит, что метки задаются в виде целых чисел, и в данном случае для функции потерь нужно использовать SparseCategoricalCrossentropy.

Если же label_mode определен как categorical, то метки задаются в виде вектора с прямым унитарным кодом, и в таком случае для функции потери нужно использовать CategoricalCrossentropy.

Если label_mode определен как binary, то метка задается как одно из всего двух возможных значений, и тогда для функции потери нужно использовать BinaryCrossentropy

Использование TPU

In [ ]:
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="local")
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
#тут пишется и компилируется модель. Я не знаю почему, но у меня проблемы с TPU в google collab, хотя код всё ещё актуален.
# т.к. не для огромных моделей прирост по сравнению с gpu незначительный, оставлю разбор использования tpu на потом

Архитектуры CNN

In [3]:
vgg16_model = tf.keras.applications.vgg16.VGG16(include_top=True, # сохранение трёх полносвязных слоёв
                                                input_shape=(224, 224, 3),
                                                classes=1000,
                                                classifier_activation="softmax")

vgg19_model = tf.keras.applications.vgg19.VGG19(include_top=True,
                                                input_shape=(224, 224, 3),
                                                classes=1000,
                                                classifier_activation="softmax")

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
574710816/574710816 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [4]:
resnet_model = tf.keras.applications.resnet.ResNet101(include_top=True,
                                                      input_shape=(224, 224, 3),
                                                      classes=1000)

179648224/179648224 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [6]:
inception_model = tf.keras.applications.inception_v3.InceptionV3(include_top=True,
                                                                input_shape=(299, 299, 3),
                                                                classes=1000,
                                                                classifier_activation="softmax")


96112376/96112376 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Xception - усовершенствованная inception, которая является очень эффективной и широкоиспользуемой